## Concepts

From one current user find best correlated user

Recommend 5 best rated anime from the best correlated user

Use the best rated anime from current user and recommend 5 more 

Total 10 recommendation is provided 

## Import Library

In [2]:
import pandas as pd
import string
import re

## Variables

In [3]:
target_user_id = 2
target_vote_number = 1000
min_vote_number = 10

## Loading Dataset
▪ Loading anime.csv and rating.csv into DataFrame

In [4]:
anime_data = pd.read_csv('anime.csv')
#animeDf = animeDf[:5000]
anime_data = pd.DataFrame(anime_data)
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [5]:
rating_data = pd.read_csv('rating.csv')
rating_data = rating_data[:1000000]
rating_data = pd.DataFrame(rating_data)
rating_data

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


# Data Preprocessing

## Change variable type

In [6]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [7]:
anime_data['episodes'] = anime_data['episodes'].replace('Unknown', -1)
anime_data = anime_data.astype({"name":"string","genre":"string","type":"string","episodes":"int64"})

In [8]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  string 
 2   genre     12232 non-null  string 
 3   type      12269 non-null  string 
 4   episodes  12294 non-null  int64  
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), string(3)
memory usage: 672.5 KB


In [9]:
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


## Rename variable

In [10]:
anime_data = anime_data.rename(columns={"rating": "average_rating"})
anime_data.head(3)

,anime_id,name,genre,type,episodes,average_rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262


In [11]:
rating_data = rating_data.rename(columns={"rating": "user_rating"})
rating_data.head(3)

,user_id,anime_id,user_rating
0,1,20,-1
1,1,24,-1
2,1,79,-1


## Check Missing Value

In [12]:
rating_data.isna().sum()

user_id        0
anime_id       0
user_rating    0
dtype: int64

In [13]:
anime_data.isna().sum()

anime_id            0
name                0
genre              62
type               25
episodes            0
average_rating    230
members             0
dtype: int64

## Drop Missing Value

In [14]:
anime_data.dropna(axis=0, inplace= True)
anime_data.isna().sum()

anime_id          0
name              0
genre             0
type              0
episodes          0
average_rating    0
members           0
dtype: int64

## Check Duplicates

In [15]:
#check duplicate anime
duplicated_anime = anime_data[anime_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_anime}')

count of duplicated anime: 0


In [16]:
#check duplicate rating
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_rating}')

count of duplicated anime: 1


In [17]:
rating_data.drop_duplicates(keep='first',inplace=True)
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated anime after removing: {duplicated_rating}')

count of duplicated anime after removing: 0


## Merge Dataset by anime_id

In [18]:
raw_merged_data = pd.merge(rating_data, anime_data, on='anime_id', sort = True)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
7813605,53492,34475,5,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813606,70547,34475,9,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813607,70768,34475,-1,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813608,48766,34476,-1,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51


## Drop Unnessary Column

In [19]:
#members are not nessesary
#raw_merged_data = raw_merged_data.drop('members', axis=1)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
7813605,53492,34475,5,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813606,70547,34475,9,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813607,70768,34475,-1,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813608,48766,34476,-1,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51


## Drop Unnessary Row
Drop the row that less than min_vote_number

In [20]:
#get number of rating for each anime
grouped_data = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
#get the anime that fulfilled the constaint which is more than min_vote_number
trimmed_data = grouped_data[grouped_data['user_rating'] > min_vote_number]
#trim the data
raw_merged_data = raw_merged_data.loc[raw_merged_data['anime_id'].isin(trimmed_data.index.values.tolist())]
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
7813585,62948,34325,-1,Pokemon XY&amp;Z Specials,"Action, Adventure, Comedy, Fantasy, Kids",Special,2,7.05,1454
7813586,68797,34325,6,Pokemon XY&amp;Z Specials,"Action, Adventure, Comedy, Fantasy, Kids",Special,2,7.05,1454
7813587,69964,34325,8,Pokemon XY&amp;Z Specials,"Action, Adventure, Comedy, Fantasy, Kids",Special,2,7.05,1454
7813588,70900,34325,7,Pokemon XY&amp;Z Specials,"Action, Adventure, Comedy, Fantasy, Kids",Special,2,7.05,1454


## Clean Text

In [21]:
#before clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [22]:
def clean_text(text):
    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    text = re.sub('[^a-zA-Z]','', text)
  
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [23]:
raw_merged_data['name']=raw_merged_data['name'].apply(clean_text)
anime_data['name'] = anime_data['name'].apply(clean_text)

C:\Users\Eng Lip\AppData\Local\Temp\ipykernel_32552\1514506093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_merged_data['name']=raw_merged_data['name'].apply(clean_text)


In [24]:
#after clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# PART 1: User Based Correlation

## Find rating_anime_matrix

In [25]:
rating_anime_matrix = raw_merged_data.pivot_table(index='user_id', columns=['anime_id'], values = 'user_rating', fill_value= -1)
rating_anime_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,33979,34015,34085,34103,34107,34238,34240,34283,34324,34325
user_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,8,-1,-1,6,-1,6,6,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Find anime_rating_matrix

In [26]:
anime_rating_matrix = rating_anime_matrix.transpose()
anime_rating_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
anime_id,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.0,-1.0,10.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,8.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,10.0,-1.0
6,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10.0,-1.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Find the information of target_user_id

In [50]:
anime_rating_user = anime_rating_matrix[target_user_id]
pd.DataFrame(anime_rating_user.sort_values(ascending= False)).rename(columns={target_user_id: f"user_id {target_user_id}'s rating"})

,user_id 2's rating
anime_id,
11771,10.0
1,-1.0
10334,-1.0
10370,-1.0
10361,-1.0
...,...
2873,-1.0
2872,-1.0
2871,-1.0


## Find Correlation between target_user_id with all the other users

In [28]:
similar_users = anime_rating_matrix.corrwith(anime_rating_user).dropna()
similar_users

user_id
1       -0.000261
2        1.000000
3        0.131319
5       -0.002620
7       -0.002453
           ...   
73512   -0.000447
73513   -0.000744
73514   -0.000131
73515   -0.001756
73516   -0.000185
Length: 69598, dtype: float64

## Format into Data Frame

In [29]:
sorted_similar_users = pd.DataFrame(similar_users, columns=['correlation']).sort_values(ascending= False, by= 'correlation')
sorted_similar_users

,correlation
user_id,
671,1.000000
13514,1.000000
62578,1.000000
34995,1.000000
36998,1.000000
...,...
51562,-0.005924
59406,-0.006118
23247,-0.006185


## Find the best correlated user

In [30]:
#most similar user is the user that at index number 1
most_similar_user = sorted_similar_users.index.values
most_similar_user = most_similar_user[1]
most_similar_user

13514

## Find 5 best rated anime from the user that have highest correlation value

In [31]:
user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)
user_recommendation = user_recommendation[:5]

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
5523135,13514,11771,7,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315
1906729,13514,1195,-1,Zero no Tsukaima,"Action, Adventure, Comedy, Ecchi, Fantasy, Har...",TV,13,7.62,346828
2363208,13514,1840,-1,Zero no Tsukaima Futatsuki no Kishi,"Action, Adventure, Comedy, Ecchi, Fantasy, Har...",TV,12,7.73,235684
3130155,13514,3712,-1,Zero no Tsukaima Princesses no Rondo,"Action, Adventure, Comedy, Ecchi, Fantasy, Har...",TV,12,7.60,210891
4855285,13514,9919,-1,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823


In [32]:
user_recommendation_list = user_recommendation['anime_id'].tolist()
user_recommendation_list = user_recommendation_list

[11771, 1195, 1840, 3712, 9919, 14407, 16592, 16894, 18507, 19429]

# PART 2: Item Based Correlation

## Use the user's highest rated anime to recommend other related anime

In [33]:
target_user_rating = rating_data.loc[rating_data['user_id'] == target_user_id].sort_values(by = 'user_rating', ascending= False)
pd.DataFrame(target_user_rating.head(10))

,user_id,anime_id,user_rating
153,2,11771,10
154,2,12189,-1
155,2,16417,-1


In [34]:
top_rated_anime_id = target_user_rating['anime_id'].tolist()[0]
top_rated_anime_id

11771

In [35]:
ratings = rating_anime_matrix[top_rated_anime_id]
pd.DataFrame(ratings).rename(columns={top_rated_anime_id: f"{top_rated_anime_id}'s rating"})

,11771's rating
user_id,
1,-1
2,10
3,10
4,-1
5,-1
...,...
73512,-1
73513,-1
73514,-1


In [36]:
similar_animes = rating_anime_matrix.corrwith(ratings)
pd.DataFrame(similar_animes, columns=['correlation'])

,correlation
anime_id,
1,0.037895
5,0.003360
6,0.041805
7,0.004984
8,0.024586
...,...
34238,0.037488
34240,0.072956
34283,0.021463


## Identify the most correlated anime

In [37]:
sorted_similar_anime = pd.DataFrame(similar_animes, columns=['correlation']).sort_values(by= 'correlation', ascending= False)
sorted_similar_anime

,correlation
anime_id,
11771,1.000000
16894,0.803475
24415,0.602449
20583,0.471563
16916,0.376617
...,...
615,-0.014778
870,-0.015375
208,-0.015503


## Eliminate the source anime

In [38]:
sorted_similar_anime = sorted_similar_anime[1:]
sorted_similar_anime

,correlation
anime_id,
16894,0.803475
24415,0.602449
20583,0.471563
16916,0.376617
28891,0.357931
...,...
615,-0.014778
870,-0.015375
208,-0.015503


## Ensure the identified anime is popular
Higher number of votes means more popular

In [39]:
#Get number of rating for each anime
rating_votes = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
rating_votes.rename(columns={'user_rating': 'rating_count'})

,rating_count
anime_id,
1,15509
5,6927
6,11077
7,2629
8,413
...,...
34238,39
34240,1546
34283,20


In [40]:
similar_animes_ratings = sorted_similar_anime.join(rating_votes['user_rating']).sort_values(by = 'correlation', ascending = False)
similar_animes_ratings

,correlation,user_rating
anime_id,,
16894,0.803475,8235
24415,0.602449,4902
20583,0.471563,7369
16916,0.376617,2017
28891,0.357931,3975
...,...,...
615,-0.014778,431
870,-0.015375,658
208,-0.015503,2261


## Get the anime that have higher votes and have higher correlation

In [41]:
similar_popular_animes = similar_animes_ratings.loc[similar_animes_ratings['user_rating']>=target_vote_number].dropna()
similar_popular_animes

,correlation,user_rating
anime_id,,
16894,0.803475,8235
24415,0.602449,4902
20583,0.471563,7369
16916,0.376617,2017
28891,0.357931,3975
...,...,...
564,-0.013417,1363
1303,-0.013633,2618
208,-0.015503,2261


In [42]:
most_similar_popular_animes = similar_popular_animes[:5]
most_similar_popular_animes_list = most_similar_popular_animes.index.to_list()
most_similar_popular_animes_list

[16894, 24415, 20583, 16916, 28891]

# Result Showing

In [43]:
final_recommendation_result = user_recommendation_list.copy()
final_recommendation_result += most_similar_popular_animes_list.copy()
final_recommendation_result

[11771,
 1195,
 1840,
 3712,
 9919,
 14407,
 16592,
 16894,
 18507,
 19429,
 16894,
 24415,
 20583,
 16916,
 28891]

## Show Target User's Rating

In [44]:
target_user = rating_data.loc[rating_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)
pd.DataFrame(target_user.head(10))

,user_id,anime_id,user_rating
153,2,11771,10
154,2,12189,-1
155,2,16417,-1


## Show Target Anime's information 

In [45]:
target_anime = anime_data.loc[anime_data['anime_id'] == top_rated_anime_id]
pd.DataFrame(target_anime)

,anime_id,name,genre,type,episodes,average_rating,members
122,11771,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315


## Show Final Recommendation Result

In [46]:
recommended_anime = anime_data.loc[anime_data['anime_id'].isin(final_recommendation_result)]
pd.DataFrame(recommended_anime)

,anime_id,name,genre,type,episodes,average_rating,members
14,28891,Haikyuu Second Season,"Comedy, Drama, School, Shounen, Sports",TV,25,8.93,179342
43,20583,Haikyuu,"Comedy, Drama, School, Shounen, Sports",TV,25,8.68,284498
58,24415,Kuroko no Basket 3rd Season,"Comedy, School, Shounen, Sports",TV,25,8.62,184525
72,16894,Kuroko no Basket 2nd Season,"Comedy, School, Shounen, Sports",TV,25,8.58,243325
122,11771,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315
443,16916,Kuroko no Basket Tip Off,"Comedy, School, Shounen, Sports",Special,1,8.07,41760
643,9919,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823
800,14407,Persona 3 the Movie 1 Spring of Birth,"Action, Fantasy, Seinen, Supernatural",Movie,1,7.83,45568
1006,1840,Zero no Tsukaima Futatsuki no Kishi,"Action, Adventure, Comedy, Ecchi, Fantasy, Har...",TV,12,7.73,235684
1115,18507,Free,"Comedy, School, Slice of Life, Sports",TV,12,7.67,265791


## Evaluation

In [1]:
#do not have actual value how?
from sklearn.metrics import mean_squared_error
import math
y_actual = [1,2,3,4,5]
y_predicted = [1.6,2.5,2.9,3,4.1]
 
MSE = mean_squared_error(y_actual, y_predicted)
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.6971370023173351
